# Relation of museuns in Toronto
### Download de DataSet

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
from sklearn.cluster import KMeans
import folium 
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors

In [81]:
List_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(List_url).text

In [82]:
soup = BeautifulSoup(source, 'xml')

In [83]:
table=soup.find('table')

In [84]:
column_names=['Postalcode','Borough','Neighbourhood']
df = pd.DataFrame(columns=column_names)

In [85]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [86]:

df.head()

,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Removing the Not Assigned

In [87]:
df = df.drop(df[df.Neighbourhood == "Not assigned"].index)

In [88]:
df.head()

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Building the Toronto Map

In [91]:
def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [92]:
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [93]:
geo_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geo_merged = pd.merge(geo_df, df_merge, on='Postalcode')

toronto_data=geo_merged[['Postalcode','Borough','Neighbourhood','Latitude','Longitude']]
toronto_data

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


In [94]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [95]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Get the venues Location and category in Toronto with query = Museum

In [96]:
CLIENT_ID = '3NR4YVMXG3TDBXAX3IIJ43LIDOSLDOMDE2MTUU2RAP0FJPRZ' # your Foursquare ID
CLIENT_SECRET = 'YXNZQVDHOJDZUMAYG5E3ZOFMOONODUEVKZDJTCQ0UO34UGCS' # your Foursquare Secret
VERSION = '20180604'

In [97]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    qur = 'Museum'
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            qur,
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [100]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

In [104]:
toronto_venues

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
1,Don Mills,43.725900,-79.340923,Moriyama Nikkei Heritage Centre,43.726494,-79.334830,History Museum
2,"Regent Park, Harbourfront",43.654260,-79.360636,Enoch Turner Schoolhouse,43.652873,-79.361672,History Museum
3,"Garden District, Ryerson",43.657162,-79.378937,Friar’s Music Museum,43.655662,-79.380426,Museum
4,"Garden District, Ryerson",43.657162,-79.378937,Dance Collection Danse,43.653924,-79.375918,Art Museum
5,"Garden District, Ryerson",43.657162,-79.378937,Elgin Theatre Loading Dock,43.652728,-79.378368,Art Museum
6,St. James Town,43.651494,-79.375418,Toronto Sculpture Garden,43.649895,-79.373715,Art Gallery
7,St. James Town,43.651494,-79.375418,Ontario Heritage Trust,43.650490,-79.378020,Office
8,Berczy Park,43.644771,-79.373306,Hockey Hall Of Fame (Hockey Hall of Fame),43.646974,-79.377323,Museum
9,Berczy Park,43.644771,-79.373306,Redpath Sugar,43.643356,-79.370450,Building


### Creating a Map of Toronto with the location of the venues

In [103]:
# create map of Toronto using latitude and longitude values
map_toronto_venues = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_venues['Venue Latitude'], toronto_venues['Venue Longitude'], toronto_venues['Venue'], toronto_venues['Venue Category']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_venues)  
    
map_toronto_venues

# Breve analise dos Dados

### Venues count by Neighbourhood

In [177]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,3,3,3,3,3,3
Central Bay Street,3,3,3,3,3,3
"Commerce Court, Victoria Hotel",3,3,3,3,3,3
"Del Ray, Mount Dennis, Keelsdale and Silverthorn",1,1,1,1,1,1
Don Mills,1,1,1,1,1,1
"First Canadian Place, Underground city",3,3,3,3,3,3
"Garden District, Ryerson",3,3,3,3,3,3
"Harbourfront East, Union Station, Toronto Islands",3,3,3,3,3,3
"Kensington Market, Chinatown, Grange Park",2,2,2,2,2,2


### One hot encoding

In [178]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

In [179]:
toronto_onehot.insert(loc=0, column='Neighbourhood', value=toronto_venues['Neighbourhood'] )
toronto_onehot.shape

(38, 7)

In [180]:
toronto_onehot

,Neighbourhood,Art Gallery,Art Museum,Building,History Museum,Museum,Office
0,"Rouge Hill, Port Union, Highland Creek",0,0,0,1,0,0
1,Don Mills,0,0,0,1,0,0
2,"Regent Park, Harbourfront",0,0,0,1,0,0
3,"Garden District, Ryerson",0,0,0,0,1,0
4,"Garden District, Ryerson",0,1,0,0,0,0
5,"Garden District, Ryerson",0,1,0,0,0,0
6,St. James Town,1,0,0,0,0,0
7,St. James Town,0,0,0,0,0,1
8,Berczy Park,0,0,0,0,1,0
9,Berczy Park,0,0,1,0,0,0


In [181]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Art Gallery,Art Museum,Building,History Museum,Museum,Office
0,Berczy Park,0.0,0.333333,0.333333,0.000000,0.333333,0.000000
1,Central Bay Street,0.0,0.333333,0.000000,0.333333,0.333333,0.000000
2,"Commerce Court, Victoria Hotel",0.0,0.000000,0.000000,0.333333,0.333333,0.333333
3,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",0.0,0.000000,0.000000,0.000000,1.000000,0.000000
4,Don Mills,0.0,0.000000,0.000000,1.000000,0.000000,0.000000


In [182]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Most common venues in Toronto by Neighbourhood

In [183]:
num_top_venues = 6

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
0,Berczy Park,Museum,Building,Art Museum,Office,History Museum,Art Gallery
1,Central Bay Street,Museum,History Museum,Art Museum,Office,Building,Art Gallery
2,"Commerce Court, Victoria Hotel",Office,Museum,History Museum,Building,Art Museum,Art Gallery
3,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",Museum,Office,History Museum,Building,Art Museum,Art Gallery
4,Don Mills,History Museum,Office,Museum,Building,Art Museum,Art Gallery
5,"First Canadian Place, Underground city",Office,Museum,History Museum,Building,Art Museum,Art Gallery
6,"Garden District, Ryerson",Art Museum,Museum,Office,History Museum,Building,Art Gallery
7,"Harbourfront East, Union Station, Toronto Islands",History Museum,Office,Museum,Building,Art Museum,Art Gallery
8,"Kensington Market, Chinatown, Grange Park",Museum,History Museum,Office,Building,Art Museum,Art Gallery
9,"Queen's Park, Ontario Provincial Government",History Museum,Office,Museum,Building,Art Museum,Art Gallery


In [184]:
# set number of clusters
kclusters = 2

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 0, 1, 1, 0, 1, 0])

In [185]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [186]:
neighborhoods_venues_sorted

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
0,1,Berczy Park,Museum,Building,Art Museum,Office,History Museum,Art Gallery
1,1,Central Bay Street,Museum,History Museum,Art Museum,Office,Building,Art Gallery
2,1,"Commerce Court, Victoria Hotel",Office,Museum,History Museum,Building,Art Museum,Art Gallery
3,1,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",Museum,Office,History Museum,Building,Art Museum,Art Gallery
4,0,Don Mills,History Museum,Office,Museum,Building,Art Museum,Art Gallery
5,1,"First Canadian Place, Underground city",Office,Museum,History Museum,Building,Art Museum,Art Gallery
6,1,"Garden District, Ryerson",Art Museum,Museum,Office,History Museum,Building,Art Gallery
7,0,"Harbourfront East, Union Station, Toronto Islands",History Museum,Office,Museum,Building,Art Museum,Art Gallery
8,1,"Kensington Market, Chinatown, Grange Park",Museum,History Museum,Office,Building,Art Museum,Art Gallery
9,0,"Queen's Park, Ontario Provincial Government",History Museum,Office,Museum,Building,Art Museum,Art Gallery


In [187]:
toronto_merged = toronto_data

In [188]:
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0.0,History Museum,Office,Museum,Building,Art Museum,Art Gallery
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,M1G,Scarborough,Woburn,43.770992,-79.216917,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [189]:
toronto_merged['Cluster Labels'].fillna(0, inplace=True)

In [190]:
toronto_merged
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].apply(np.int64)

In [191]:
toronto_merged

,Postalcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,0,NaN,NaN,NaN,NaN,NaN,NaN
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0,History Museum,Office,Museum,Building,Art Museum,Art Gallery
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,NaN,NaN,NaN,NaN,NaN,NaN
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,NaN,NaN,NaN,NaN,NaN,NaN
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188,0,NaN,NaN,NaN,NaN,NaN,NaN
99,M9P,Etobicoke,Westmount,43.696319,-79.532242,0,NaN,NaN,NaN,NaN,NaN,NaN
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724,0,NaN,NaN,NaN,NaN,NaN,NaN
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437,0,NaN,NaN,NaN,NaN,NaN,NaN


## Removendo os bairros que não possuem museus

In [192]:
toronto_merged = toronto_merged.dropna()

In [193]:
toronto_merged

,Postalcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0,History Museum,Office,Museum,Building,Art Museum,Art Gallery
26,M3B,North York,Don Mills,43.745906,-79.352188,0,History Museum,Office,Museum,Building,Art Museum,Art Gallery
27,M3C,North York,Don Mills,43.725900,-79.340923,0,History Museum,Office,Museum,Building,Art Museum,Art Gallery
53,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,History Museum,Office,Museum,Building,Art Museum,Art Gallery
54,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Art Museum,Museum,Office,History Museum,Building,Art Gallery
55,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Office,Art Gallery,Museum,History Museum,Building,Art Museum
56,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Museum,Building,Art Museum,Office,History Museum,Art Gallery
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1,Museum,History Museum,Art Museum,Office,Building,Art Gallery
58,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,1,Museum,History Museum,Art Museum,Office,Building,Art Gallery
59,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,0,History Museum,Office,Museum,Building,Art Museum,Art Gallery


In [194]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [195]:
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


In [196]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters